In [1]:
import sys
sys.path.append('/mnt/lovit/git/soynlp/')

import soynlp
import konlpy

print('soynlp version = {}'.format(soynlp.__version__))
print('konlpy version = {}'.format(konlpy.__version__))

soynlp version = 0.0.46
konlpy version = 0.5.1


In [2]:
corpus_path = '/mnt/lovit/works/fastcampus_text_ml/2nd/data/corpus_10days/news/2016-10-20_article_all_normed.txt'
index_path = '/mnt/lovit/works/fastcampus_text_ml/2nd/data/corpus_10days/news/2016-10-20_article_all.index'

In [14]:
from soynlp.utils import DoublespaceLineCorpus

index = [doc for doc in DoublespaceLineCorpus(index_path)]
categories = [doc.split('\t')[1] for doc in index]

# 106 entertrain
document_idx = {i for i, category in enumerate(categories) if category == '106'}
docs = [doc for doc in DoublespaceLineCorpus(corpus_path)]
entertain_sents = [sent.strip() for i, doc in enumerate(docs) for sent in doc.split('  ') if i in document_idx]
ioi_sents = [sent.strip() for doc in docs for sent in doc.split('  ') if '아이오아이' in doc]

print('entertain sents = {}'.format(len(entertain_sents)))
print('I.O.I sents = {}'.format(len(ioi_sents)))

entertain sents = 45678
I.O.I sents = 769


In [20]:
import re
number_pattern = re.compile('[0-9]+')

def has_number(word):
    return len(number_pattern.findall(word)) > 0

print(has_number('1234번호'))
print(has_number('글자입니다'))

True
False


In [15]:
from soynlp.noun import LRNounExtractor_v2

noun_extractor = LRNounExtractor_v2()
nouns = noun_extractor.train_extract(ioi_sents)

[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=1260, neg=1173, common=12
[Noun Extractor] counting eojeols
[Eojeol Counter] n eojeol = 4396 from 769 sents. mem=1.051 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. mem=1.051 Gb
[Noun Extractor] batch prediction was completed for 1815 words
[Noun Extractor] checked compounds. discovered 0 compounds
[Noun Extractor] postprocessing detaching_features : 615 -> 615
[Noun Extractor] postprocessing ignore_features : 615 -> 605
[Noun Extractor] postprocessing ignore_NJ : 605 -> 567
[Noun Extractor] 567 nouns (0 compounds) with min count=1
[Noun Extractor] flushing was done. mem=1.051 Gb                    
[Noun Extractor] 44.11 % eojeols are covered


In [21]:
nouns = {noun:score for noun, score in nouns.items() if len(noun) > 1 and not has_number(noun)}

In [22]:
len(nouns)

468

In [23]:
nouns

{'가능': NounScore(frequency=2, score=1.0),
 '가득': NounScore(frequency=3, score=1.0),
 '가사': NounScore(frequency=14, score=1.0),
 '가수': NounScore(frequency=14, score=1.0),
 '가수들': NounScore(frequency=2, score=1.0),
 '가요계': NounScore(frequency=12, score=1.0),
 '가운데': NounScore(frequency=6, score=1.0),
 '가창력': NounScore(frequency=6, score=1.0),
 '각각': NounScore(frequency=7, score=1.0),
 '각국': NounScore(frequency=2, score=1.0),
 '각오': NounScore(frequency=13, score=1.0),
 '간주': NounScore(frequency=2, score=1.0),
 '감사': NounScore(frequency=15, score=1.0),
 '감성': NounScore(frequency=11, score=1.0),
 '감수성': NounScore(frequency=2, score=1.0),
 '강렬': NounScore(frequency=23, score=1.0),
 '강점': NounScore(frequency=2, score=1.0),
 '개성': NounScore(frequency=2, score=1.0),
 '개최': NounScore(frequency=1, score=0.5),
 '거쳐': NounScore(frequency=4, score=1.0),
 '걸그룹': NounScore(frequency=86, score=1.0),
 '걸그룹들': NounScore(frequency=3, score=1.0),
 '걸크러쉬': NounScore(frequency=27, score=1.0),
 '검증': NounScor

In [31]:
with open('./user_nouns_nnp.txt', 'w', encoding='utf-8') as f:
    for noun in sorted(nouns):
        f.write('{}\tNNP\n'.format(noun))

with open('./user_nouns_nng.txt', 'w', encoding='utf-8') as f:
    for noun in sorted(nouns):
        f.write('{}\tNNG\n'.format(noun))

In [32]:
from konlpy.tag import Komoran

basic_komoran = Komoran(max_heap_size=2048)
my_komoran = Komoran(userdic='./user_nouns_nng.txt')
my_komoran_nnp = Komoran(userdic='./user_nouns_nnp.txt')

In [35]:
print(my_komoran.pos('김청하가 김청하한다'))
print(my_komoran.pos('김청하가 김청하했다'))

[('김청하', 'NNG'), ('가', 'JKS'), ('김청하', 'NNG'), ('하', 'XSV'), ('ㄴ다', 'EC')]
[('김청하', 'NNG'), ('가', 'JKS'), ('김청하', 'NNG'), ('하', 'XSV'), ('았', 'EP'), ('다', 'EC')]


In [36]:
print(my_komoran_nnp.pos('김청하가 김청하한다'))
print(my_komoran_nnp.pos('김청하가 김청하했다'))

[('김청하', 'NNP'), ('가', 'JKS'), ('김청하', 'NNP'), ('한', 'NNP'), ('다', 'JX')]
[('김청하', 'NNP'), ('가', 'JKS'), ('김청하', 'NNP'), ('하', 'XSV'), ('았', 'EP'), ('다', 'EC')]


In [47]:
print(my_komoran.pos('트와이스의 우아하게를 틀어줘'))
print(my_komoran_nnp.pos('트와이스의 우아하게를 틀어줘'))

[('트와이스', 'NNG'), ('의', 'JKG'), ('우아', 'XR'), ('하', 'XSA'), ('게', 'EC'), ('를', 'JKO'), ('틀', 'VV'), ('어', 'EC'), ('주', 'VX'), ('어', 'EC')]
[('트와이스', 'NNP'), ('의', 'JKG'), ('우아', 'XR'), ('하', 'XSA'), ('게', 'EC'), ('를', 'JKO'), ('틀', 'VV'), ('어', 'EC'), ('주', 'VX'), ('어', 'EC')]


In [30]:
for i, sent in enumerate(ioi_sents):

    if len(sent) > 100:
        continue

    basic_words = ' '.join(basic_komoran.morphs(sent))
    withdic_words = ' '.join(my_komoran.morphs(sent))

    basic_pos = ' '.join(basic_komoran.pos(sent, join=True))
    withdic_pos = ' '.join(my_komoran.pos(sent, join=True))

    if basic_words == withdic_words:
        continue

    print('#{}'.format(i))
    print('sent     : {}'.format(sent))
    print('basic    : {}'.format(basic_pos))
    print('user dic : {}\n'.format(withdic_pos))

#1
sent     : 신인 걸그룹 불독이 아이오아이 와 비슷한 시기 활동하게 된 소감을 밝혔다
basic    : 신인/NNG 걸/VV ㄹ/ETM 그룹/NNP 불독/NNG 이/JKS 아이오/NNP 아이/NNP 오/VV 아/EC 비슷/XR 하/XSA ㄴ/ETM 시기/NNG 활동/NNG 하/XSV 게/EC 되/VV ㄴ/ETM 소감/NNG 을/JKO 밝히/VV 었/EP 다/EC
user dic : 신인/NNG 걸그룹/NNG 불독/NNG 이/JKS 아이오아이/NNG 오/VV 아/EC 비슷/XR 하/XSA ㄴ/ETM 시기/NNG 활동/NNG 하/XSV 게/EC 되/VV ㄴ/ETM 소감/NNG 을/JKO 밝히/VV 었/EP 다/EC

#2
sent     : 20일 오전 서울 마포구 홍대 롤링홀에서 불독 데뷔 앨범 어때요 발매 기념 쇼케이스가 진행됐다
basic    : 20/SN 일/NNB 오전/NNP 서울/NNP 마포구/NNP 홍대/NNP 롤링/NNG 홀/NNG 에서/JKB 불독/NNG 데뷔/NNP 앨범/NNP 어떻/VA 어요/EC 발매/NNG 기념/NNG 쇼케이스/NNP 가/JKS 진행/NNG 되/XSV 었/EP 다/EC
user dic : 20/SN 일/NNB 오전/NNP 서울/NNP 마포구/NNP 홍대/NNP 롤링홀/NNG 에서/JKB 불독/NNG 데뷔/NNG 앨범/NNG 어떻/VA 어요/EC 발매/NNG 기념/NNG 쇼케이스/NNG 가/JKS 진행/NNG 되/XSV 었/EP 다/EC

#3
sent     : 쇼케이스 간담회에서 불독은 아이오아이와 비슷한 시기 활동하며 선의의 경쟁을 벌이게 된 데 대한 감회를 드러냈다
basic    : 쇼케이스/NNP 간담회/NNG 에서/JKB 불독/NNG 은/JX 아이오/NNP 아이와/NNP 비슷/XR 하/XSA ㄴ/ETM 시기/NNG 활동/NNG 하/XSV 며/EC 선의/NNP 의/JKG 경쟁/NNG 을/JKO 벌이/VV 게/EC 되/VV ㄴ/ETM 데/NNB 대하/VV ㄴ/ETM 감회/NNG 를/JKO 드러내/VV 

#57
sent     : 엠카운트다운 아이오아이 신용재 컴백 한층 업그레이드된 모습 공개
basic    : 엠카운트다운/NNP 아이오/NNP 아이/NNP 신용재/NNP 컴백/NNG 한층/MAG 업그레이드/NNG 되/XSV ㄴ/ETM 모습/NNG 공개/NNG
user dic : 엠카운트다운/NNP 아이오아이/NNG 신용재/NNG 컴백/NNG 한층/MAG 업그레이드/NNG 되/XSV ㄴ/ETM 모습/NNG 공개/NNG

#58
sent     : 엠카운트다운 에서 아이오아이와 신용재가 컴백 무대를 선보인다
basic    : 엠카운트다운/NNP 에서/JKB 아이오/NNP 아이와/NNP 신용재/NNP 가/JKS 컴백/NNG 무대/NNG 를/JKO 선보이/VV ㄴ다/EC
user dic : 엠카운트다운/NNP 에서/JKB 아이오아이/NNG 와/JC 신용재/NNG 가/JKS 컴백/NNG 무대/NNG 를/JKO 선보이/VV ㄴ다/EC

#59
sent     : 20일 방송되는 엠넷 엠카운트다운 에서는 아이오아이와 신용재의 컴백 무대를 공개 ㅎㅏㅆ다
basic    : 20/SN 일/NNB 방송/NNG 되/XSV 는/ETM 엠넷/NNP 엠카운트다운/NNP 에서/JKB 는/JX 아이오/NNP 아이와/NNP 신용재/NNP 의/JKG 컴백/NNG 무대/NNG 를/JKO 공개/NNG 핬다/NA
user dic : 20/SN 일/NNB 방송/NNG 되/XSV 는/ETM 엠넷/NNG 엠카운트다운/NNP 에서/JKB 는/JX 아이오아이/NNG 와/JC 신용재/NNG 의/JKG 컴백/NNG 무대/NNG 를/JKO 공개/NNG 핬다/NA

#68
sent     : 엠카운트다운 걸그룹 아이오아이 가 너무너무너무 로 컴백했다
basic    : 엠카운트다운/NNP 걸/VV ㄹ/ETM 그룹/NNP 아이오/NNP 아이/NNP 가/VV 아/EC 너무너무/MAG 너무/MAG 로/NNG 컴백/NNG 하/XSV 았/EP 다/EC
user dic : 엠카운트다운/NNP 걸그룹/NNG 아이오아이/N

#138
sent     : 다이아 소속사는 나머지 멤버들은 개인 음악 활동 및 예능 활동을 이어갈 예정이며 멤버 희현과 예빈은 22일 본격오감만족 라이브를 통해 다양한 음악을 선보인다 고 밝혔다
basic    : 다이아/NNP 소속사/NNG 는/JX 나머지/NNG 멤버/NNG 들/XSN 은/JX 개인/NNG 음악/NNG 활동/NNG 및/MAJ 예능/NNG 활동/NNG 을/JKO 잇/VV 어/EC 가/VX ㄹ/ETM 예정/NNG 이/VCP 며/EC 멤버/NNP 희현과/NA 예비/NNG ㄴ/JX 은/JX 22/SN 일/NNB 본격/NNG 오/NNP 감/NNG 만족/NNP 라이브/NNP 를/JKO 통하/VV 아/EC 다양/XR 하/XSA ㄴ/ETM 음악/NNG 을/JKO 선보이/VV ㄴ다/EC 고/MM 밝히/VV 었/EP 다/EC
user dic : 다이아/NNP 소속사/NNG 는/JX 나머지/NNG 멤버들/NNG 은/JX 개인/NNG 음악/NNG 활동/NNG 및/MAJ 예능/NNG 활동/NNG 을/JKO 잇/VV 어/EC 가/VX ㄹ/ETM 예정/NNG 이/VCP 며/EC 멤버/NNG 희현과/NA 예비/NNG ㄴ/JX 은/JX 22/SN 일/NNB 본격/NNG 오/NNP 감/NNG 만족/NNP 라이브/NNP 를/JKO 통하/VV 아/EC 다양/XR 하/XSA ㄴ/ETM 음악/NNG 을/JKO 선보이/VV ㄴ다/EC 고/MM 밝히/VV 었/EP 다/EC

#142
sent     : 엔터온뉴스 이예은 기자 20일 오후 아이오아이부터 방탄소년단까지 다양한 라인업으로 엠카운트다운 이하 엠카 무대가 꾸며진다
basic    : 엔터/NNG 오/VV ㄴ/ETM 뉴스/NNP 이예은/NNP 기자/NNG 20/SN 일/NNB 오후/NNG 아이오/NNP 아이/NNP 부터/JX 방탄소년단/NNP 까지/JX 다양/XR 하/XSA ㄴ/ETM 라인업/NNP 으로/JKB 엠카운트다운/NNP 이하/NNG 엠카/NA 무대/NNG 가/JKS 꾸미/VV 어/EC 지/VX ㄴ다/EC
user 

#300
sent     : 콘셉트는 걸크러시 다 소라는 걸크러시 하면 떠오르는 포미닛 2 1 선배님을 본받고 싶다 며 훗날에는 선배님들을 뛰어 넘어 월드투어를 하는 게 목표 라고 당찬 포부를 밝혔다
basic    : 콘셉트/NNP 는/JX 걸/VV ㄹ/ETM 크러시/NNP 다/MAG 소라/NNP 는/JX 걸/VV ㄹ/ETM 크러시/NNP 하/VV 면/EC 떠오르/VV 는/ETM 포미닛/NNP 2/SN 1/SN 선배/NNG 님/XSN 을/JKO 본받/VV 고/EC 싶/VX 다/EC 이/VCP 며/EC 훗날/NNG 에/JKB 는/JX 선배/NNG 님/XSN 들/XSN 을/JKO 뛰/VV 어/EC 넘/VV 어/EC 월드/NNP 투어/NNG 를/JKO 하/VV 는/ETM 게/NNG 목표/NNG 라고/NNP 당차/VA ㄴ/ETM 포부/NNG 를/JKO 밝히/VV 었/EP 다/EC
user dic : 콘셉트/NNG 는/JX 걸/VV ㄹ/ETM 크러시/NNP 다/MAG 소라/NNG 는/JX 걸/VV ㄹ/ETM 크러시/NNP 하/VV 면/EC 떠오르/VV 는/ETM 포미닛/NNG 2/SN 1/SN 선배님/NNG 을/JKO 본받/VV 고/EC 싶/VX 다/EC 이/VCP 며/EC 훗날/NNG 에/JKB 는/JX 선배님들/NNG 을/JKO 뛰/VV 어/EC 넘/VV 어/EC 월드/NNP 투어/NNG 를/JKO 하/VV 는/ETM 게/NNG 목표/NNG 라고/NNP 당차/VA ㄴ/ETM 포부/NNG 를/JKO 밝히/VV 었/EP 다/EC

#301
sent     : 긴 연습생 기간을 거쳐 데뷔의 꿈을 이룬 불독은 어때요 로 독 하게 활동할 각오다
basic    : 길/VA ㄴ/ETM 연습/NNG 생/XSN 기간/NNG 을/JKO 거치/VV 어/EC 데뷔/NNP 의/JKG 꿈/NNG 을/JKO 이루/VV ㄴ/ETM 불독/NNG 은/JX 어떻/VA 어요/EC 로/NNG 독/NNP 하/VV 게/EC 활동/NNG 하/XSV ㄹ/ETM 각오/NNG 다/JX
user dic : 길/

#379
sent     : 한편 지난 17일 공개된 아이오아이의 두 번째 미니앨범 에는 타이틀곡 너무너무너무 를 비롯하여 잠깐만 내 말대로 해줘 음 어쩌면 등 다섯 개의 곡이 수록되어 있다
basic    : 한편/NNG 지나/VV ㄴ/ETM 17/SN 일/NNB 공개/NNG 되/XSV ㄴ/ETM 아이오/NNP 아이/NNP 의/JKG 두/MM 번/NNB 째/XSN 미니/NNP 앨범/NNP 에/NNG 는/JX 타이틀/NNP 곡/NNG 너무너무/MAG 너무/MAG 를/JKO 비롯/XR 하/XSA 아/EC 잠깐/NNP 만/JX 내/NP 말/NNG 대로/JX 하/VV 아/EC 주/VX 어/EC 음/NNG 어쩌면/MAG 등/NNB 다섯/NR 개/NNB 의/JKG 곡/NNG 이/JKS 수록/NNG 되/XSV 어/EC 있/VV 다/EC
user dic : 한편/NNG 지나/VV ㄴ/ETM 17/SN 일/NNB 공개/NNG 되/XSV ㄴ/ETM 아이오아이/NNG 의/JKG 두/MM 번째/NNG 미니앨범/NNG 에/NNG 는/JX 타이틀곡/NNG 너무너무너무/NNG 를/JKO 비롯/NNG 하/XSV 아/EC 잠깐/NNP 만/JX 내/NP 말/NNG 대로/JX 하/VV 아/EC 주/VX 어/EC 음/NNG 어쩌면/MAG 등/NNB 다섯/NR 개/NNB 의/JKG 곡/NNG 이/JKS 수록/NNG 되/XSV 어/EC 있/VV 다/EC

#390
sent     : 마이데일리 최지예 기자 프로젝트 걸그룹 아이오아이가 나흘째 음원차트 정상을 수성하고 있다
basic    : 마이데일리/NNP 최/XPN 지/NNG 예/NNG 기자/NNG 프로젝트/NNP 걸/VV ㄹ/ETM 그룹/NNP 아이오/NNP 아이/NNP 가/JKS 나흘/NNG 째/XSN 음원/NNP 차트/NNP 정상/NNG 을/JKO 수성/NNG 하/XSV 고/EC 있/VV 다/EC
user dic : 마이데일리/NNP 최/XPN 지/NNG 예/NNG 기자/NNG 프로젝트/NNG 걸그룹/NNG 아이오아이/NNG 가/JKS 나

#452
sent     : 완전체로 뭉친 이날 아이오아이는 상큼하고 발랄한 매력으로 눈길을 끌었다 박진영의 손을 거친 아이오아이는 퍼포먼스와 비주얼 모두 업그레이드 돼 팬들의 뜨거운 호응을 얻었다
basic    : 완전체/NNP 로/JKB 뭉치/VV ㄴ/ETM 이날/NNG 아이오/NNP 아이/NNP 는/JX 상큼/XR 하/XSA 고/EC 발랄/XR 하/XSA ㄴ/ETM 매력/NNP 으로/JKB 눈길/NNG 을/JKO 끌/VV 었/EP 다/EC 박진영/NNP 의/JKG 손/NNG 을/JKO 거치/VV ㄴ/ETM 아이오/NNP 아이/NNP 는/JX 퍼포먼스/NNP 와/JC 비주얼/NNG 모두/MAG 업그레이드/NNP 되/VV 어/EC 팬/NNG 들/XSN 의/JKG 뜨겁/VA ㄴ/ETM 호응/NNG 을/JKO 얻/VV 었/EP 다/EC
user dic : 완전체/NNG 로/JKB 뭉치/VV ㄴ/ETM 이날/NNG 아이오아이/NNG 는/JX 상큼/NNG 하/XSV 고/EC 발랄/NNG 하/XSV ㄴ/ETM 매력/NNG 으로/JKB 눈길/NNG 을/JKO 끌/VV 었/EP 다/EC 박진영/NNG 의/JKG 손/NNG 을/JKO 거치/VV ㄴ/ETM 아이오아이/NNG 는/JX 퍼포먼스/NNG 와/JC 비주얼/NNG 모두/MAG 업그레이드/NNP 되/VV 어/EC 팬들/NNG 의/JKG 뜨겁/VA ㄴ/ETM 호응/NNG 을/JKO 얻/VV 었/EP 다/EC

#461
sent     : 방탄소년단은 20일 방송된 엠넷 엠카운트다운 에서 정규 2집 앨범 윙즈 타이틀곡 피 땀 눈물 로 정상에 올랐다 함께 후보에 오른 여성듀오 다비치를 제친 결과다
basic    : 방탄소년단/NNP 은/JX 20/SN 일/NNB 방송/NNG 되/XSV ㄴ/ETM 엠넷/NNP 엠카운트다운/NNP 에서/JKB 정규/NNP 2/SN 집/NNG 앨범/NNP 윙즈/NA 타이틀/NNP 곡/NNG 피/NNG 땀/NNG 눈물/NNG 로/NNG 정상/NNG 에/JKB 오르/VV 았/EP 다/EC 함께/

#510
sent     : 이날 방탄소년단은 멤버들의 부모님 모두 너무 감사합니다 라며 모든 분들의 관심과 사랑 덕에 받게 된 상 이라고 우승 소감을 전했다
basic    : 이날/NNG 방탄소년단/NNP 은/JX 멤버/NNG 들/XSN 의/JKG 부모/NNG 님/XSN 모두/MAG 너무/MAG 감사/NNG 하/XSV ㅂ니다/EC 이/VCP 라며/EC 모든/MM 분/NNB 들/XSN 의/JKG 관심/NNG 과/JC 사랑/NNG 덕/NNG 에/JKB 받/VV 게/EC 되/VV ㄴ/ETM 상/NNG 이/NP 이/VCP 라고/EC 우승/NNP 소감/NNG 을/JKO 전하/VV 았/EP 다/EC
user dic : 이날/NNG 방탄소년단/NNG 은/JX 멤버들/NNG 의/JKG 부모님/NNG 모두/MAG 너무/MAG 감사/NNG 하/XSV ㅂ니다/EC 이/VCP 라며/EC 모든/MM 분들/NNG 의/JKG 관심/NNG 과/JC 사랑/NNG 덕/NNG 에/JKB 받/VV 게/EC 되/VV ㄴ/ETM 상/NNG 이/NP 이/VCP 라고/EC 우승/NNP 소감/NNG 을/JKO 전하/VV 았/EP 다/EC

#511
sent     : 앞서 방탄소년단은 만약 1위를 차지한다면 섹시한 콘셉트로 무대를 소화하겠다 고 공약을 내건 바 있다 이에 그들은 앙코르 무대에서 섹시 버전의 앵콜송을 열창해 눈길을 끌었다
basic    : 앞서/MAG 방탄소년단/NNP 은/JX 만약/MAG 1/SN 위/NNB 를/JKO 차지/NNG 하/XSV ㄴ다면/EC 섹시/XR 하/XSA ㄴ/ETM 콘셉트/NNP 로/JKB 무대/NNG 를/JKO 소화/NNG 하/XSV 겠/EP 다/EC 고/MM 공약/NNG 을/JKO 내걸/VV ㄴ/ETM 바/NNB 있/VV 다/EC 이/NP 에/JKB 그/NP 들/XSN 은/JX 앙코르/NNP 무대/NNG 에서/JKB 섹시/NNP 버전/NNP 의/JKG 앵콜/NNG 송/NNP 을/JKO 열창/NNG 하/XSV 아/EC 눈길/NNG 을/JKO 끌/VV 었/EP 다

user dic : 한편/NNG 걸그룹/NNG 불독/NNG 은/JX 데뷔/NNG 싱글/NNG 타이틀곡/NNG 어떻/VA 어요/EC 로/NNG 본격적/NNG 이/VCP ㄴ/ETM 활동/NNG 에/JKB 돌입/NNG 하/XSV ㄹ/ETM 예정/NNG 이/VCP 다/EC

#588
sent     : 트와이스는 오는 24일 세 번째 미니앨범 1 로 돌아온다 타이틀곡 는 작곡가 블랙아이드필승의 곡으로 트와이스의 색다른 변신을 보여줄 신선하고 상큼한 곡이 될 전망이다
basic    : 트와이스/NNP 는/JX 오/VV 는/ETM 24/SN 일/NNB 세/MM 번/NNB 째/XSN 미니/NNP 앨범/NNP 1/SN 로/NNG 돌아오/VV ㄴ다/EC 타이틀/NNP 곡/NNG 늘/VV ㄴ/ETM 작곡가/NNP 블랙/NNP 아/NNP 이드/NNP 필승/NNG 의/JKG 곡/NNG 으로/JKB 트와이스/NNP 의/JKG 색다르/VA ㄴ/ETM 변신/NNG 을/JKO 보이/VV 어/EC 주/VX ㄹ/ETM 신선/XR 하/XSA 고/EC 상큼/XR 하/XSA ㄴ/ETM 곡/NNG 이/JKS 되/VV ㄹ/ETM 전망/NNG 이/VCP 다/EC
user dic : 트와이스/NNG 는/JX 오/VV 는/ETM 24/SN 일/NNB 세/MM 번째/NNG 미니앨범/NNG 1/SN 로/NNG 돌아오/VV ㄴ다/EC 타이틀곡/NNG 늘/VV ㄴ/ETM 작곡가/NNP 블랙/NNP 아/NNP 이드/NNP 필승/NNG 의/JKG 곡/NNG 으로/JKB 트와이스/NNG 의/JKG 색다르/VA ㄴ/ETM 변신/NNG 을/JKO 보이/VV 어/EC 주/VX ㄹ/ETM 신선/XR 하/XSA 고/EC 상큼/NNG 하/XSV ㄴ/ETM 곡/NNG 이/JKS 되/VV ㄹ/ETM 전망/NNG 이/VCP 다/EC

#590
sent     : 대한민국 아이돌 히트제조기 박진영 그의 감독하에 탄생한 트와이스 그리고 히트곡 선물을 받은 아이오아이까지 그들이 과연 이번 활동에서 몇 개의 트로피를 수집할지 귀추가 주목되

#652
sent     : 일산 뉴스1스타 권현진 기자 아이오아이 주결경 왼쪽 이 19일 오후 경기도 고양시 일산동구 장항동 드림센터에서 열린 뮤직 쇼 챔피언 쇼챔 에 출연해 화려한 공연을 선보이고 있다
basic    : 일산/NNP 뉴스1/NNP 스타/NNP 권/NNP 현지/NNG ㄴ/JX 기자/NNG 아이오/NNP 아이/NNP 주/NNG 결/NNG 경/XSN 왼쪽/NNG 이/MM 19/SN 일/NNB 오후/NNG 경기도/NNP 고양시/NNP 일산동구/NNP 장항동/NNP 드림/NNP 센터/NNP 에서/JKB 열리/VV ㄴ/ETM 뮤직/NNP 쇼 챔피언/NNP 쇼/NNG 채/VV ㅁ/ETN 에/JKB 출연/NNG 하/XSV 아/EC 화려/XR 하/XSA ㄴ/ETM 공연/NNG 을/JKO 선보이/VV 고/EC 있/VV 다/EC
user dic : 일산/NNP 뉴스1/NNP 스타/NNP 권/NNP 현지/NNG ㄴ/JX 기자/NNG 아이오아이/NNG 주/NNG 결/NNG 경/XSN 왼쪽/NNG 이/MM 19/SN 일/NNB 오후/NNG 경기도/NNP 고양시/NNP 일산동구/NNP 장항동/NNP 드림/NNP 센터/NNP 에서/JKB 열리/VV ㄴ/ETM 뮤직/NNG 쇼 챔피언/NNP 쇼/NNG 채/VV ㅁ/ETN 에/JKB 출연/NNG 하/XSV 아/EC 화려/NNG 하/XSV ㄴ/ETM 공연/NNG 을/JKO 선보이/VV 고/EC 있/VV 다/EC

#655
sent     : 일산 뉴스1스타 권현진 기자 아이오아이 유연이 19일 오후 경기도 고양시 일산동구 장항동 드림센터에서 열린 뮤직 쇼 챔피언 쇼챔 에 출연해 화려한 공연을 선보이고 있다
basic    : 일산/NNP 뉴스1/NNP 스타/NNP 권/NNP 현지/NNG ㄴ/JX 기자/NNG 아이오/NNP 아이/NNP 유연/NNP 이/JKS 19/SN 일/NNB 오후/NNG 경기도/NNP 고양시/NNP 일산동구/NNP 장항동/NNP 드림/NNP 센터/NNP 에서/JKB 열리/VV ㄴ/ETM 뮤직/NNP 쇼 

#722
sent     : 서울 뉴스1스타 유수경 기자 아이오아이가 물오른 미모와 상큼함을 자랑했다
basic    : 서울/NNP 뉴스1/NNP 스타/NNP 유수/NNP 경/NNP 기자/NNG 아이오/NNP 아이/NNP 가/JKS 물오르/VV ㄴ/ETM 미모/NNG 와/JC 상큼/XR 하/XSA ㅁ/ETN 을/JKO 자랑/NNG 하/XSV 았/EP 다/EC
user dic : 서울/NNP 뉴스1/NNP 스타/NNP 유수/NNP 경/NNP 기자/NNG 아이오아이/NNG 가/JKS 물오르/VV ㄴ/ETM 미모/NNG 와/JC 상큼/NNG 하/XSV ㅁ/ETN 을/JKO 자랑/NNG 하/XSV 았/EP 다/EC

#723
sent     : 아이오아이는 20일 오후 6시 방송된 엠넷 엠카운트다운 에서 컴백무대를 선보였다
basic    : 아이오/NNP 아이/NNP 는/JX 20/SN 일/NNB 오후/NNG 6/SN 시/NNB 방송/NNG 되/XSV ㄴ/ETM 엠넷/NNP 엠카운트다운/NNP 에서/JKB 컴백/NNG 무대/NNG 를/JKO 선보이/VV 었/EP 다/EC
user dic : 아이오아이/NNG 는/JX 20/SN 일/NNB 오후/NNG 6/SN 시/NNB 방송/NNG 되/XSV ㄴ/ETM 엠넷/NNG 엠카운트다운/NNP 에서/JKB 컴백무대/NNG 를/JKO 선보이/VV 었/EP 다/EC

#724
sent     : 이날 아이오아이는 너무너무너무 무대를 통해 상큼 발랄 귀여운 매력을 과시했다 깜찍한 표정과 몸짓이 남성 팬들의 마음을 뒤흔들었다
basic    : 이날/NNG 아이오/NNP 아이/NNP 는/JX 너무너무/MAG 너무/MAG 무대/NNG 를/JKO 통하/VV 아/EC 상큼/XR 발랄/XR 귀엽/VA ㄴ/ETM 매력/NNG 을/JKO 과시/NNG 하/XSV 았/EP 다/EC 깜찍/XR 하/XSA ㄴ/ETM 표정/NNG 과/JC 몸짓/NNG 이/JKS 남성/NNP 팬/NNG 들/XSN 의/JKG 마음/NNG 을/JKO 뒤흔들/VV 었/EP